In [86]:
import os
from scipy import stats
import pandas as pd
import numpy as np

In [87]:
odir="/mnt/lab_data2/anusri/variant-scorer/src/output/CAGI/output/"
dirs = os.listdir(odir)

enformer = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/results/variant_data/CAGI/enformer/allGroupsBenchmark.txt", sep="\t", header=0)
enformer.head()

,Locus,deltaSVM,Enf.CAGE.Agnostic,Enf.CAGE,Enf.DNase.Agnostic,Enf.DNase,Enformer,G3,G7,dSVM LASSO,dSVM RF,dSVM (1 DHS),G5,CADD,Inter-replicate
0,F9,0.483518,0.245,0.252,0.573,0.291,0.572,0.624,0.391,0.557,0.566,0.489,0.564,-0.086,0.88
1,GP1BB,NaN,0.370,0.469,0.555,0.564,0.563,0.556,0.566,0.261,0.371,0.421,0.348,-0.083,0.88
2,HBB,NaN,0.491,0.596,0.550,0.578,0.600,0.446,0.416,0.445,0.328,0.393,0.392,0.049,0.77
3,HBG1,NaN,0.178,0.302,0.104,0.189,0.474,0.571,0.591,0.397,0.439,0.342,0.479,0.174,0.92
4,HNF4A,0.091578,0.173,0.156,0.170,0.288,0.335,0.339,0.397,0.225,0.151,0.134,0.191,0.111,0.89


In [88]:
cagi_df_list = []

for idir in dirs: 
    idir=str(idir)
    ifile=odir+idir+"/summary.mean.variant_scores_new_2.tsv"
    if os.path.isfile(ifile):
        if not ifile.endswith("tsv"):
            continue
        data = pd.read_csv(ifile, sep="\t", header=0)
        #print(data.head())
        data[["chr_temp", "pos_temp", "ref_temp", "alt_temp", "label", "Confidence", "challenge"]] = data["rsid"].str.split(",", expand=True)
        #print(data.columns)
        data["sign_jsd.mean"] = data["jsd.mean"] * np.sign(data["logfc.mean"])
        data["model_used"] = idir
        data = data[['chr', 'pos', 'allele1', 'allele2', 'rsid', 'logfc.mean', "sign_jsd.mean", 'abs_logfc_x_jsd_x_max_percentile.mean',
       'abs_logfc_x_jsd_x_max_percentile.mean.pval', "label", "Confidence", "challenge", "model_used"]]
        cagi_df_list.append(data)

cagi_df = pd.concat(cagi_df_list)

In [89]:
cagi_df.head()

,chr,pos,allele1,allele2,rsid,logfc.mean,sign_jsd.mean,abs_logfc_x_jsd_x_max_percentile.mean,abs_logfc_x_jsd_x_max_percentile.mean.pval,label,Confidence,challenge,model_used
0,chr6,37775290,G,A,"6,37775290,G,A,0.03,0.01,challenge_ZFAND3.tsv",-0.143521,-0.024822,0.001164,0.043634,0.03,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
1,chr6,37775290,G,C,"6,37775290,G,C,0.03,0.01,challenge_ZFAND3.tsv",-0.131439,-0.023676,0.000977,0.047589,0.03,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
2,chr6,37775290,G,T,"6,37775290,G,T,0.07,0.01,challenge_ZFAND3.tsv",-0.086450,-0.023025,0.000608,0.067393,0.07,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
3,chr6,37775291,T,A,"6,37775291,T,A,-0.07,0.01,challenge_ZFAND3.tsv",-0.013275,-0.027508,0.000358,0.125360,-0.07,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
4,chr6,37775291,T,C,"6,37775291,T,C,0.03,0.01,challenge_ZFAND3.tsv",0.033312,0.020898,0.000256,0.150673,0.03,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas


In [90]:
odir = "/mnt/lab_data2/anusri/chrombpnet/results/variant_data/CAGI/data/"
dirs = os.listdir(odir)

In [91]:
enf_dfs= []
for idir in dirs:
    idir=str(idir)
    ifile=odir+idir
    if os.path.isfile(ifile):
            #print(ifile)
#             if idir in ["challenge_MSMB.tsv"]: # MSMB ref and alt also dont match
#                 continue
            if not ifile.endswith("tsv"):
                continue
            if not idir.startswith("challenge"):
                continue
            print(idir)
            orig = pd.read_csv(ifile, sep="\t", header=None, comment="#")
            orig[0] = "chr"+orig[0].astype(str)
            orig.columns = ["chr_hg19", "pos_hg19", "ref", "alt", "label", "Confidence"]
            ifile1 = "/mnt/lab_data2/anusri/chrombpnet/results/variant_data/CAGI/enformer_preds/CAGI5_train_test_hg19_to_hg38_liftover/"
            liftoverd = ifile1 + "withChr."+idir
            lft_data = pd.read_csv(liftoverd, sep="\t", header=None)
            lft_data.columns = ["chr_hg38", "pos_hg38", "ref_hg38", "alt_hg38", "label_hg38", "Confidence_hg38"]
            assert(lft_data.shape[0]==orig.shape[0])
            
            combined = pd.concat([orig, lft_data],axis=1)
            combined["challenge"] = idir
            #print(combined.head())
            
#             assert((combined["ref"]==combined["ref_hg38"]).all())
#             assert((combined["alt"]==combined["alt_hg38"]).all())
            assert((combined["label"]==combined["label_hg38"]).all())
            assert((combined["Confidence"]==combined["Confidence_hg38"]).all())
            
            enf_dfs.append(combined)
            

challenge_HBG1.tsv
challenge_PKLR.tsv
challenge_MYCrs6983267.tsv
challenge_TERT-GBM.tsv
challenge_MSMB.tsv
challenge_F9.tsv
challenge_IRF6.tsv
challenge_HNF4A.tsv
challenge_IRF4.tsv
challenge_SORT1.tsv
challenge_ZFAND3.tsv
challenge_HBB.tsv
challenge_LDLR.tsv
challenge_TERT-HEK293T.tsv
challenge_GP1BB.tsv


In [92]:
enf_df = pd.concat(enf_dfs)
enf_df.head(1)

,chr_hg19,pos_hg19,ref,alt,label,Confidence,chr_hg38,pos_hg38,ref_hg38,alt_hg38,label_hg38,Confidence_hg38,challenge
0,chr11,5271035,G,A,0.19,0.06,chr11,5249805,G,A,0.19,0.06,challenge_HBG1.tsv


In [93]:
cagi_df

,chr,pos,allele1,allele2,rsid,logfc.mean,sign_jsd.mean,abs_logfc_x_jsd_x_max_percentile.mean,abs_logfc_x_jsd_x_max_percentile.mean.pval,label,Confidence,challenge,model_used
0,chr6,37775290,G,A,"6,37775290,G,A,0.03,0.01,challenge_ZFAND3.tsv",-0.143521,-0.024822,0.001164,0.043634,0.03,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
1,chr6,37775290,G,C,"6,37775290,G,C,0.03,0.01,challenge_ZFAND3.tsv",-0.131439,-0.023676,0.000977,0.047589,0.03,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
2,chr6,37775290,G,T,"6,37775290,G,T,0.07,0.01,challenge_ZFAND3.tsv",-0.086450,-0.023025,0.000608,0.067393,0.07,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
3,chr6,37775291,T,A,"6,37775291,T,A,-0.07,0.01,challenge_ZFAND3.tsv",-0.013275,-0.027508,0.000358,0.125360,-0.07,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
4,chr6,37775291,T,C,"6,37775291,T,C,0.03,0.01,challenge_ZFAND3.tsv",0.033312,0.020898,0.000256,0.150673,0.03,0.01,challenge_ZFAND3.tsv,ENCSR056PFI_pancreas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,chr6,37775848,G,C,"6,37775848,G,C,0.19,0.03,challenge_ZFAND3.tsv",0.012688,0.011428,0.000087,0.362118,0.19,0.03,challenge_ZFAND3.tsv,ENCSR539WBA_pancreas
1243,chr6,37775848,G,T,"6,37775848,G,T,0.02,0.0,challenge_ZFAND3.tsv",-0.014364,-0.010755,0.000096,0.424178,0.02,0.0,challenge_ZFAND3.tsv,ENCSR539WBA_pancreas
1244,chr6,37775849,C,A,"6,37775849,C,A,0.0,0.0,challenge_ZFAND3.tsv",-0.009251,-0.012372,0.000121,0.369296,0.0,0.0,challenge_ZFAND3.tsv,ENCSR539WBA_pancreas
1245,chr6,37775849,C,G,"6,37775849,C,G,-0.02,0.01,challenge_ZFAND3.tsv",0.007366,0.012662,0.000069,0.434610,-0.02,0.01,challenge_ZFAND3.tsv,ENCSR539WBA_pancreas


In [94]:
enf_df["label"] = enf_df["label"].astype(float)
enf_df["Confidence"] = enf_df["Confidence"].astype(float)
cagi_df["label"] = cagi_df["label"].astype(float)
cagi_df["Confidence"] = cagi_df["Confidence"].astype(float)
enf_df["pos_hg19"] = enf_df["pos_hg19"].astype(int)
cagi_df["pos"] = cagi_df["pos"].astype(int)

cagi_df_new = cagi_df.merge(enf_df, left_on=["chr", "pos", "allele1", "allele2", "label", "Confidence", "challenge"], right_on=["chr_hg19", "pos_hg19", "ref", "alt", "label", "Confidence", "challenge"], how="left", validate="many_to_one")


In [95]:
cagi_df_new.shape

(28422, 23)

In [96]:
cagi_df.shape

(28422, 13)

In [97]:
cagi_df.shape

(28422, 13)

In [98]:
cagi_df_new.head(1)

,chr,pos,allele1,allele2,rsid,logfc.mean,sign_jsd.mean,abs_logfc_x_jsd_x_max_percentile.mean,abs_logfc_x_jsd_x_max_percentile.mean.pval,label,...,chr_hg19,pos_hg19,ref,alt,chr_hg38,pos_hg38,ref_hg38,alt_hg38,label_hg38,Confidence_hg38
0,chr6,37775290,G,A,"6,37775290,G,A,0.03,0.01,challenge_ZFAND3.tsv",-0.143521,-0.024822,0.001164,0.043634,0.03,...,chr6,37775290,G,A,chr6,37807514,G,A,0.03,0.01


In [99]:
sum(np.isnan(cagi_df_new["pos_hg38"]))

0

In [100]:
list(set(cagi_df_new["model_used"]))

['ENCSR056PFI_pancreas',
 'ENCSR595HZQ_pancreas',
 'ENCSR724CND_keratinocyte',
 'ENCSR000EPQ_keratinocyte',
 'ENCSR000EJR_HEK293',
 'ENCSR035RVH_keratinocyte',
 'ENCSR236KPK_pancreas',
 'K562',
 'ENCSR464TKV_pancreas',
 'HEPG2',
 'ENCSR999VDH_pancreas',
 'ENCSR000FEK_SK-MEL',
 'ENCSR539WBA_pancreas',
 'ENCSR338NEI_pancreas',
 'ENCSR242RPQ_pancreas',
 'ENCSR828FVZ_pancreas',
 'ENCSR405YLV_pancreas']

In [101]:
cagi_df_new["model_used"] = cagi_df_new["model_used"].str.replace("K562",'ENCSR000EOT_K562')
cagi_df_new["model_used"] = cagi_df_new["model_used"].str.replace("HEPG2",'ENCSR149XIL_HEPG2')


In [102]:
list(set(cagi_df_new["model_used"]))

['ENCSR056PFI_pancreas',
 'ENCSR595HZQ_pancreas',
 'ENCSR724CND_keratinocyte',
 'ENCSR000EPQ_keratinocyte',
 'ENCSR000EJR_HEK293',
 'ENCSR035RVH_keratinocyte',
 'ENCSR236KPK_pancreas',
 'ENCSR000EOT_K562',
 'ENCSR464TKV_pancreas',
 'ENCSR999VDH_pancreas',
 'ENCSR405YLV_pancreas',
 'ENCSR000FEK_SK-MEL',
 'ENCSR539WBA_pancreas',
 'ENCSR338NEI_pancreas',
 'ENCSR242RPQ_pancreas',
 'ENCSR828FVZ_pancreas',
 'ENCSR149XIL_HEPG2']

In [103]:
valid_ids = ["ENCSR178JBL_pancreas", "ENCSR464TKV_pancreas", "ENCSR828FVZ_pancreas", 
 "ENCSR539WBA_pancreas", "ENCSR405YLV_pancreas", "ENCSR595HZQ_pancreas",
 "ENCSR000ELH_keratinocyte", "ENCSR000EPQ_keratinocyte", "ENCSR035RVH_keratinocyte",
 "ENCSR724CND_keratinocyte", "ENCSR000FEK_SK-MEL", "ENCSR000EJV_HEPG2", "ENCSR149XIL_HEPG2",
 "ENCSR000EKN_K562", "ENCSR000EKP_K562", "ENCSR000EKQ_k562", "ENCSR000EOT_K562",
 "ENCSR000EJR_HEK293"]

convert_dict = {"ENCSR464TKV":"ENCSR228IKB",
                "ENCSR828FVZ":"ENCSR421KNA",
                "ENCSR539WBA":"ENCSR522WLH",
                "ENCSR405YLV":"ENCSR524OCB"}


In [104]:
import deepdish as dd
datat = dd.io.load("/mnt/lab_data2/anusri/chrombpnet/results/variant_data/CAGI/enformer_preds/sad.h5")
enformer_preds = pd.DataFrame()

enformer_preds["chr"] = datat["chr"].astype(str)
enformer_preds["pos"] = datat["pos"]
enformer_preds["ref"] = datat["ref_allele"].astype(str)
enformer_preds["alt"] = datat["alt_allele"].astype(str)

enformer_preds["rsid"] = enformer_preds["chr"].astype(str) + '_' +  enformer_preds["pos"].astype(str) + "_" + enformer_preds["ref"] + '_' +  enformer_preds["alt"]


enformer_labesls = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/results/variant_data/CAGI/enformer_preds/target_humans.txt", sep="\t", header=0)
enformer_labesls["encode_id"] = enformer_labesls["file"].str.split("/", expand=True)[8]

In [105]:
enformer_labesls

,index,genome,identifier,file,clip,scale,sum_stat,description,encode_id
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,ENCSR000EIJ
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,ENCSR000EIK
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,ENCSR000EIL
3,3,0,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,ENCSR000EIP
4,4,0,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,ENCSR000EIS
...,...,...,...,...,...,...,...,...,...
5308,5308,0,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R,CNhs14239
5309,5309,0,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...,CNhs14240
5310,5310,0,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS,CNhs14241
5311,5311,0,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727,CNhs14244


In [106]:
sad = datat["SAD"]
sar = datat["SADR"]

enformer_sad = []
enformer_sar = []

cagi_df_new_drop = cagi_df_new[cagi_df_new["model_used"].isin(valid_ids)]
num_enf_missing=0
for i,r in cagi_df_new_drop.iterrows():
    #print(r)
    rsid = r["chr_hg38"] + "_" + str(r["pos_hg38"]) + "_" + r["ref_hg38"] + "_" + r["alt_hg38"]
    #print(rsid)
    #assert(len(enformer_preds[enformer_preds["rsid"].isin([rsid])].index.values) <= 1)
    if len(enformer_preds[enformer_preds["rsid"].isin([rsid])].index.values) > 1:
        print("gtr than 1", rsid)
        index = enformer_preds[enformer_preds["rsid"].isin([rsid])].index.values[0]
        encid = r["model_used"].split("_")[0]
        if encid in convert_dict:
            encid = convert_dict[encid]

        assert(len(enformer_labesls[enformer_labesls["encode_id"]==encid]["index"].values)==1)

        #print(sad.shape)
        enformer_sad.append(sad[index,enformer_labesls[enformer_labesls["encode_id"]==encid]["index"].values[0]])
        enformer_sar.append(sar[index,enformer_labesls[enformer_labesls["encode_id"]==encid]["index"].values[0]])
        
    elif len(enformer_preds[enformer_preds["rsid"].isin([rsid])].index.values) == 1:
        index = enformer_preds[enformer_preds["rsid"].isin([rsid])].index.values[0]
        encid = r["model_used"].split("_")[0]
        if encid in convert_dict:
            encid = convert_dict[encid]

        assert(len(enformer_labesls[enformer_labesls["encode_id"]==encid]["index"].values)==1)

        #print(sad.shape)
        enformer_sad.append(sad[index,enformer_labesls[enformer_labesls["encode_id"]==encid]["index"].values[0]])
        enformer_sar.append(sar[index,enformer_labesls[enformer_labesls["encode_id"]==encid]["index"].values[0]])
    else:
        print(rsid)
        num_enf_missing+=1
        enformer_sad.append(None)
        enformer_sar.append(None)


chr1_209816391_G_A
chr1_209816391_G_C
chr1_209816391_G_T
chrX_139530462_C_A
chrX_139530462_C_G
chrX_139530462_C_T
gtr than 1 chr19_11089246_A_C
gtr than 1 chr19_11089246_A_G
gtr than 1 chr19_11089246_A_T
gtr than 1 chr19_11089247_C_A
gtr than 1 chr19_11089247_C_G
gtr than 1 chr19_11089247_C_T
gtr than 1 chr19_11089248_C_A
gtr than 1 chr19_11089248_C_G
gtr than 1 chr19_11089248_C_T
gtr than 1 chr19_11089249_C_A
gtr than 1 chr19_11089249_C_G
gtr than 1 chr19_11089249_C_T
gtr than 1 chr19_11089250_A_C
gtr than 1 chr19_11089250_A_G
gtr than 1 chr19_11089250_A_T
gtr than 1 chr19_11089251_A_C
gtr than 1 chr19_11089251_A_G
gtr than 1 chr19_11089251_A_T
gtr than 1 chr19_11089252_A_C
gtr than 1 chr19_11089252_A_G
gtr than 1 chr19_11089252_A_T
gtr than 1 chr19_11089253_T_A
gtr than 1 chr19_11089253_T_C
gtr than 1 chr19_11089253_T_G
gtr than 1 chr19_11089254_A_C
gtr than 1 chr19_11089254_A_G
gtr than 1 chr19_11089254_A_T
gtr than 1 chr19_11089255_C_A
gtr than 1 chr19_11089255_C_G
gtr than 1 chr19

gtr than 1 chr19_11089367_G_T
gtr than 1 chr19_11089368_C_A
gtr than 1 chr19_11089368_C_G
gtr than 1 chr19_11089368_C_T
gtr than 1 chr19_11089369_A_C
gtr than 1 chr19_11089369_A_G
gtr than 1 chr19_11089369_A_T
gtr than 1 chr19_11089370_G_A
gtr than 1 chr19_11089370_G_C
gtr than 1 chr19_11089370_G_T
gtr than 1 chr19_11089371_T_A
gtr than 1 chr19_11089371_T_C
gtr than 1 chr19_11089371_T_G
gtr than 1 chr19_11089372_G_A
gtr than 1 chr19_11089372_G_C
gtr than 1 chr19_11089372_G_T
gtr than 1 chr19_11089373_A_C
gtr than 1 chr19_11089373_A_G
gtr than 1 chr19_11089373_A_T
gtr than 1 chr19_11089390_A_C
gtr than 1 chr19_11089390_A_G
gtr than 1 chr19_11089390_A_T
gtr than 1 chr19_11089391_A_C
gtr than 1 chr19_11089391_A_G
gtr than 1 chr19_11089391_A_T
gtr than 1 chr19_11089392_T_A
gtr than 1 chr19_11089392_T_C
gtr than 1 chr19_11089392_T_G
gtr than 1 chr19_11089393_C_A
gtr than 1 chr19_11089393_C_G
gtr than 1 chr19_11089393_C_T
gtr than 1 chr19_11089394_A_C
gtr than 1 chr19_11089394_A_G
gtr than 1

gtr than 1 chr19_11089477_A_G
gtr than 1 chr19_11089477_A_T
gtr than 1 chr19_11089478_G_A
gtr than 1 chr19_11089478_G_C
gtr than 1 chr19_11089478_G_T
gtr than 1 chr19_11089479_C_A
gtr than 1 chr19_11089479_C_G
gtr than 1 chr19_11089479_C_T
gtr than 1 chr19_11089480_C_A
gtr than 1 chr19_11089480_C_G
gtr than 1 chr19_11089480_C_T
gtr than 1 chr19_11089481_A_C
gtr than 1 chr19_11089481_A_G
gtr than 1 chr19_11089481_A_T
gtr than 1 chr19_11089482_G_A
gtr than 1 chr19_11089482_G_C
gtr than 1 chr19_11089482_G_T
gtr than 1 chr19_11089483_G_A
gtr than 1 chr19_11089483_G_C
gtr than 1 chr19_11089483_G_T
gtr than 1 chr19_11089484_G_A
gtr than 1 chr19_11089484_G_C
gtr than 1 chr19_11089484_G_T
gtr than 1 chr19_11089485_T_A
gtr than 1 chr19_11089485_T_C
gtr than 1 chr19_11089485_T_G
gtr than 1 chr19_11089502_G_A
gtr than 1 chr19_11089502_G_C
gtr than 1 chr19_11089502_G_T
gtr than 1 chr19_11089503_C_A
gtr than 1 chr19_11089503_C_G
gtr than 1 chr19_11089503_C_T
gtr than 1 chr19_11089504_A_C
gtr than 1

gtr than 1 chr1_109274688_G_A
gtr than 1 chr1_109274688_G_C
gtr than 1 chr1_109274688_G_T
gtr than 1 chr1_109274689_A_C
gtr than 1 chr1_109274689_A_G
gtr than 1 chr1_109274689_A_T
gtr than 1 chr1_109274690_C_A
gtr than 1 chr1_109274690_C_G
gtr than 1 chr1_109274690_C_T
gtr than 1 chr1_109274691_T_A
gtr than 1 chr1_109274691_T_C
gtr than 1 chr1_109274691_T_G
gtr than 1 chr1_109274692_C_A
gtr than 1 chr1_109274692_C_G
gtr than 1 chr1_109274692_C_T
gtr than 1 chr1_109274693_A_C
gtr than 1 chr1_109274693_A_G
gtr than 1 chr1_109274693_A_T
gtr than 1 chr1_109274694_G_A
gtr than 1 chr1_109274694_G_C
gtr than 1 chr1_109274694_G_T
gtr than 1 chr1_109274695_C_A
gtr than 1 chr1_109274695_C_G
gtr than 1 chr1_109274695_C_T
gtr than 1 chr1_109274696_G_A
gtr than 1 chr1_109274696_G_C
gtr than 1 chr1_109274696_G_T
gtr than 1 chr1_109274697_C_A
gtr than 1 chr1_109274697_C_G
gtr than 1 chr1_109274697_C_T
gtr than 1 chr1_109274698_C_A
gtr than 1 chr1_109274698_C_G
gtr than 1 chr1_109274698_C_T
gtr than 1

gtr than 1 chr1_109274812_C_T
gtr than 1 chr1_109274813_C_A
gtr than 1 chr1_109274813_C_G
gtr than 1 chr1_109274813_C_T
gtr than 1 chr1_109274814_C_A
gtr than 1 chr1_109274814_C_G
gtr than 1 chr1_109274814_C_T
gtr than 1 chr1_109274815_A_C
gtr than 1 chr1_109274815_A_G
gtr than 1 chr1_109274815_A_T
gtr than 1 chr1_109274816_G_A
gtr than 1 chr1_109274816_G_C
gtr than 1 chr1_109274816_G_T
gtr than 1 chr1_109274817_G_A
gtr than 1 chr1_109274817_G_C
gtr than 1 chr1_109274817_G_T
gtr than 1 chr1_109274818_T_A
gtr than 1 chr1_109274818_T_C
gtr than 1 chr1_109274818_T_G
gtr than 1 chr1_109274819_G_A
gtr than 1 chr1_109274819_G_C
gtr than 1 chr1_109274819_G_T
gtr than 1 chr1_109274820_T_A
gtr than 1 chr1_109274820_T_C
gtr than 1 chr1_109274820_T_G
gtr than 1 chr1_109274821_T_A
gtr than 1 chr1_109274821_T_C
gtr than 1 chr1_109274821_T_G
gtr than 1 chr1_109274822_T_A
gtr than 1 chr1_109274822_T_C
gtr than 1 chr1_109274822_T_G
gtr than 1 chr1_109274823_G_A
gtr than 1 chr1_109274823_G_C
gtr than 1

...

gtr than 1 chr5_1295187_A_G
gtr than 1 chr5_1295187_A_T
gtr than 1 chr5_1295188_A_C
gtr than 1 chr5_1295188_A_G
gtr than 1 chr5_1295188_A_T
gtr than 1 chr5_1295189_G_A
gtr than 1 chr5_1295189_G_C
gtr than 1 chr5_1295189_G_T
gtr than 1 chr5_1295190_G_A
gtr than 1 chr5_1295190_G_C
gtr than 1 chr5_1295190_G_T
gtr than 1 chr5_1295191_G_A
gtr than 1 chr5_1295191_G_C
gtr than 1 chr5_1295191_G_T
gtr than 1 chr5_1295192_G_A
gtr than 1 chr5_1295192_G_C
gtr than 1 chr5_1295192_G_T
gtr than 1 chr5_1295193_C_A
gtr than 1 chr5_1295193_C_G
gtr than 1 chr5_1295193_C_T
gtr than 1 chr5_1295194_A_C
gtr than 1 chr5_1295194_A_G
gtr than 1 chr5_1295194_A_T
gtr than 1 chr5_1295195_G_A
gtr than 1 chr5_1295195_G_C
gtr than 1 chr5_1295195_G_T
gtr than 1 chr5_1295196_G_A
gtr than 1 chr5_1295196_G_C
gtr than 1 chr5_1295196_G_T
gtr than 1 chr5_1295197_A_C
gtr than 1 chr5_1295197_A_G
gtr than 1 chr5_1295197_A_T
gtr than 1 chr5_1295198_C_A
gtr than 1 chr5_1295198_C_G
gtr than 1 chr5_1295198_C_T
gtr than 1 chr5_1295

gtr than 1 chr1_155301464_T_A
gtr than 1 chr1_155301464_T_C
gtr than 1 chr1_155301464_T_G
gtr than 1 chr1_155301465_T_A
gtr than 1 chr1_155301465_T_C
gtr than 1 chr1_155301465_T_G
gtr than 1 chr1_155301466_A_G
gtr than 1 chr1_155301466_A_T
gtr than 1 chr1_155301467_T_A
gtr than 1 chr1_155301467_T_C
gtr than 1 chr1_155301467_T_G
gtr than 1 chr1_155301468_C_A
gtr than 1 chr1_155301468_C_G
gtr than 1 chr1_155301468_C_T
gtr than 1 chr1_155301469_T_A
gtr than 1 chr1_155301469_T_C
gtr than 1 chr1_155301469_T_G
gtr than 1 chr1_155301470_A_C
gtr than 1 chr1_155301470_A_G
gtr than 1 chr1_155301470_A_T
gtr than 1 chr1_155301471_A_C
gtr than 1 chr1_155301471_A_G
gtr than 1 chr1_155301471_A_T
gtr than 1 chr1_155301472_G_A
gtr than 1 chr1_155301472_G_C
gtr than 1 chr1_155301472_G_T
gtr than 1 chr1_155301473_G_A
gtr than 1 chr1_155301473_G_C
gtr than 1 chr1_155301473_G_T
gtr than 1 chr1_155301474_G_A
gtr than 1 chr1_155301474_G_C
gtr than 1 chr1_155301474_G_T
gtr than 1 chr1_155301475_A_C
gtr than 1

gtr than 1 chr1_155301590_T_A
gtr than 1 chr1_155301590_T_C
gtr than 1 chr1_155301590_T_G
gtr than 1 chr1_155301591_C_A
gtr than 1 chr1_155301591_C_G
gtr than 1 chr1_155301591_C_T
gtr than 1 chr1_155301592_C_A
gtr than 1 chr1_155301592_C_G
gtr than 1 chr1_155301592_C_T
gtr than 1 chr1_155301593_T_A
gtr than 1 chr1_155301593_T_C
gtr than 1 chr1_155301593_T_G
gtr than 1 chr1_155301594_G_A
gtr than 1 chr1_155301594_G_C
gtr than 1 chr1_155301594_G_T
gtr than 1 chr1_155301595_A_C
gtr than 1 chr1_155301595_A_G
gtr than 1 chr1_155301595_A_T
gtr than 1 chr1_155301596_T_A
gtr than 1 chr1_155301596_T_C
gtr than 1 chr1_155301596_T_G
gtr than 1 chr1_155301597_A_C
gtr than 1 chr1_155301597_A_G
gtr than 1 chr1_155301597_A_T
gtr than 1 chr1_155301598_A_C
gtr than 1 chr1_155301598_A_G
gtr than 1 chr1_155301598_A_T
gtr than 1 chr1_155301599_A_C
gtr than 1 chr1_155301599_A_G
gtr than 1 chr1_155301599_A_T
gtr than 1 chr1_155301600_T_A
gtr than 1 chr1_155301600_T_C
gtr than 1 chr1_155301600_T_G
gtr than 1

gtr than 1 chr1_155301715_G_C
gtr than 1 chr1_155301715_G_T
gtr than 1 chr1_155301716_A_C
gtr than 1 chr1_155301716_A_G
gtr than 1 chr1_155301716_A_T
gtr than 1 chr1_155301717_G_A
gtr than 1 chr1_155301717_G_C
gtr than 1 chr1_155301717_G_T
gtr than 1 chr1_155301718_C_A
gtr than 1 chr1_155301718_C_G
gtr than 1 chr1_155301718_C_T
gtr than 1 chr1_155301719_A_C
gtr than 1 chr1_155301719_A_G
gtr than 1 chr1_155301719_A_T
gtr than 1 chr1_155301720_G_A
gtr than 1 chr1_155301720_G_C
gtr than 1 chr1_155301720_G_T
gtr than 1 chr1_155301721_G_A
gtr than 1 chr1_155301721_G_C
gtr than 1 chr1_155301721_G_T
gtr than 1 chr1_155301722_G_A
gtr than 1 chr1_155301722_G_C
gtr than 1 chr1_155301722_G_T
gtr than 1 chr1_155301723_G_A
gtr than 1 chr1_155301723_G_C
gtr than 1 chr1_155301723_G_T
gtr than 1 chr1_155301724_C_A
gtr than 1 chr1_155301724_C_G
gtr than 1 chr1_155301724_C_T
gtr than 1 chr1_155301725_T_A
gtr than 1 chr1_155301725_T_C
gtr than 1 chr1_155301725_T_G
gtr than 1 chr1_155301726_G_A
gtr than 1

gtr than 1 chr1_155301839_A_G
gtr than 1 chr1_155301839_A_T
gtr than 1 chr1_155301840_G_A
gtr than 1 chr1_155301840_G_C
gtr than 1 chr1_155301840_G_T
gtr than 1 chr1_155301841_A_C
gtr than 1 chr1_155301841_A_G
gtr than 1 chr1_155301841_A_T
gtr than 1 chr1_155301842_G_A
gtr than 1 chr1_155301842_G_C
gtr than 1 chr1_155301842_G_T
gtr than 1 chr1_155301859_T_A
gtr than 1 chr1_155301859_T_C
gtr than 1 chr1_155301859_T_G
gtr than 1 chr1_155301860_A_C
gtr than 1 chr1_155301860_A_G
gtr than 1 chr1_155301860_A_T
gtr than 1 chr1_155301861_A_G
gtr than 1 chr1_155301861_A_T
gtr than 1 chr1_155301862_C_A
gtr than 1 chr1_155301862_C_G
gtr than 1 chr1_155301862_C_T
gtr than 1 chr1_155301863_C_A
gtr than 1 chr1_155301863_C_G
gtr than 1 chr1_155301863_C_T
gtr than 1 chr1_155301864_T_A
gtr than 1 chr1_155301864_T_C
chr1_155301865_G_A
chr1_155301865_G_C
chr1_155301865_G_T


In [107]:
#datat.keys()

In [108]:
cagi_df_new_drop["SAD"] = enformer_sad
cagi_df_new_drop["SAR"] = enformer_sar


/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [109]:
sum(np.isnan(cagi_df_new_drop["SAD"]))

29

In [110]:
cagi_df_new_drop.to_csv("../final_figures_new/cagi.mpra.per_model.scores.tsv", sep="\t", index=False, header=True)

In [82]:
enformer_preds[enformer_preds["rsid"].isin(["chr1_155301817_C_G"])]

,chr,pos,ref,alt,rsid
19161,chr1,155301817,C,G,chr1_155301817_C_G
20568,chr1,155301817,C,G,chr1_155301817_C_G


In [83]:
sad[19161,0]

-0.005596

In [84]:
sad[20568,0]

-0.005596